ไฟล์นี้เกิดจากความไม่รู้ว่าสามารถใช้ library *datasets* ของ huggingface ได้ เลยมานั่งแปลงไฟล์แบบ manual เอง

In [5]:
import os
import csv
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm
from io import StringIO

# LST20

In [1]:
lst20_train_folder = "/project/lt900038-ai23tn/kaggle/nlp_pos_tagging/LST20-21/LST20/train"
lst20_eval_folder = "/project/lt900038-ai23tn/kaggle/nlp_pos_tagging/LST20-21/LST20/eval"
lst20_test_folder = "/project/lt900038-ai23tn/kaggle/nlp_pos_tagging/LST20-21/LST20/test"

In [2]:
with open("ss3-final-nlp-pos-tagging/pos_list.txt", 'r') as f:
    labels_list = eval(f.read())
len(labels_list)

16

In [12]:
lst20_train_files = glob.glob(os.path.join(lst20_train_folder, '*.txt'))
lst20_eval_files = glob.glob(os.path.join(lst20_eval_folder, '*.txt'))
lst20_test_files = glob.glob(os.path.join(lst20_test_folder, '*.txt'))

lst20_train_files += lst20_eval_files

In [18]:
len(lst20_train_files), len(lst20_test_files)

(4268, 483)

In [38]:
lst20_train_dfs = []
sentence_id = 0
length = []

for filepath in tqdm(lst20_train_files):
    with open(filepath, 'r') as f:
        content = f.read()
        for s in content.split('\n\n'):
            df = pd.read_csv(StringIO(s), sep='\t', quoting=csv.QUOTE_NONE, names=['words', 'labels', 'NER', 'CLS'])
            length.append(len(df))
            if not (df['labels'].isin(labels_list)).all():
                print(filepath)
                continue
            df['words'] = df['words'].apply(lambda x: ' ' if x == '_' else x)
            df['sentence_id'] = sentence_id
            lst20_train_dfs.append(df[['sentence_id', 'words', 'labels']])
            sentence_id += 1

100%|██████████| 4268/4268 [02:04<00:00, 34.32it/s]


In [25]:
len(lst20_train_dfs)

73198

In [29]:
lst20_train_df = pd.concat(lst20_train_dfs)
len(lst20_train_df)

2955586

In [30]:
lst20_val_dfs = []

for filepath in tqdm(lst20_test_files):
    with open(filepath, 'r') as f:
        content = f.read()
        for s in content.split('\n\n'):
            df = pd.read_csv(StringIO(s), sep='\t', quoting=csv.QUOTE_NONE, names=['words', 'labels', 'NER', 'CLS'])
            if not (df['labels'].isin(labels_list)).all():
                print(filepath)
                continue
            df['words'] = df['words'].apply(lambda x: ' ' if x == '_' else x)
            df['sentence_id'] = sentence_id
            lst20_val_dfs.append(df[['sentence_id', 'words', 'labels']])
            sentence_id += 1

100%|██████████| 483/483 [00:11<00:00, 42.14it/s]


In [31]:
lst20_val_df = pd.concat(lst20_val_dfs)
len(lst20_val_df)

207278

In [35]:
os.makedirs('csv_data', exist_ok=True)

lst20_train_df.to_csv('csv_data/lst20_train.csv', index=False)
lst20_val_df.to_csv('csv_data/lst20_val.csv', index=False)